In [ ]:
%cd /Users/idawatibustan/Dev/shopee_data_science
%env PROJECT_PATH /Users/idawatibustan/Dev/shopee_data_science

In [ ]:
from datetime import datetime
import os

import pandas as pd

from model.heuristic.utilities import load_library
from model.heuristic.fashion_heuristic import run_fashion_heuristic
from utils.envs import *


In [ ]:
fashion_val = pd.read_csv(fashion_val_repo)
fashion_val.head()

In [ ]:
fashion_library = load_library('model/heuristic/fashion_library_20190316.json')

fashion_profile = fashion_library.get('primary')
fashion_profile_secondary = fashion_library.get('secondary')
fashion_max_length = fashion_library.get('length')

In [ ]:
# run heuristic function + validate accuracy on training data

fashion_train = pd.read_csv(fashion_train_repo)

start_time = datetime.now()
fashion_pred = run_fashion_heuristic(
    fashion_train, 
    fashion_profile,
    fashion_profile_secondary,
    fashion_max_length
)
duration = datetime.now() - start_time
print("duration:", duration)

# accuracy on training data
for feature in fashion_profile.keys():
    print(feature)
    """ compare actual brand and prediction on insight DF"""
    insight = fashion_train.filter(['itemid', 'title', feature])
    insight[feature+'_pred'] = fashion_pred[feature]
    total = len(insight)

    """ drop brand NA of actual brand"""
    total_known = len(insight.dropna(subset=[feature]))

    insight_accuracy = insight.dropna(subset=[feature, feature+'_pred'])
    correct_ans = insight_accuracy[feature].eq(insight_accuracy[feature+'_pred']).sum()
    total_ans = insight_accuracy[feature].count()
    print("  coverage = {:.5f}, accuracy = {:.5f}, ".format(
        total_ans/total_known,
        correct_ans/ total_ans, 
    ))


In [ ]:
# run heuristic function on submission
start_time = datetime.now()

fashion_val_pred = run_fashion_heuristic(
    fashion_val, 
    fashion_profile,
    fashion_profile_secondary,
    fashion_max_length
)

duration = datetime.now() - start_time
print(duration)

fashion_val_pred.head()

In [ ]:
# format answer for submission
fashion_val_submission = pd.DataFrame(columns=['id', 'tagging'])

for feature in fashion_profile.keys():
    temp = pd.DataFrame()
    temp['id'] = fashion_val_pred['itemid'].map(str)+"_"+feature
    temp['tagging'] = fashion_val_pred[feature]
    
    if len(fashion_val_submission) == 0:
        fashion_val_submission = temp
    else:
        fashion_val_submission = fashion_val_submission.append(temp, ignore_index=True)
    print(feature, len(fashion_val_submission))

fashion_val_submission.info()

In [ ]:
not_nan = fashion_val_submission.tagging.count()
total = len(fashion_val_submission)
print("answered = {} out of {} ({})". format(not_nan, total, not_nan/ total))

fashion_val_submission.head()

In [ ]:
import os

outpath = 'output/result'
filename = 'fashion_info_val_submission_'+datetime.now().strftime("%Y%m%d_%H%M%S")+'.csv'
print(os.path.join(outpath,filename))

fashion_val_submission.to_csv(os.path.join(outpath,filename))